Looking at the Hopkins COVID-19 data that was cloned from https://github.com/CSSEGISandData/COVID-19

In [1]:
import pandas as pd
import numpy as np
import requests
import json
from datetime import datetime as dt

# It turns out that I don't think I need the statistics library after all
# import statistics


df_confirmed = pd.read_csv('./csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv')

#df_deaths = pd.read_csv('./csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv')
#df_recovered = pd.read_csv('./csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv')

In [2]:
df_confirmed.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20
0,NaN,Thailand,15.0000,101.0000,2,3,5,7,8,8,...,47,48,50,50,50,53,59,70,75,82
1,NaN,Japan,36.0000,138.0000,2,1,2,2,4,4,...,360,420,461,502,511,581,639,639,701,773
2,NaN,Singapore,1.2833,103.8333,0,1,3,3,4,5,...,117,130,138,150,150,160,178,178,200,212
3,NaN,Nepal,28.1667,84.2500,0,0,0,1,1,1,...,1,1,1,1,1,1,1,1,1,1
4,NaN,Malaysia,2.5000,112.5000,0,0,0,3,4,4,...,50,83,93,99,117,129,149,149,197,238


In [3]:
df_confirmed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 442 entries, 0 to 441
Data columns (total 57 columns):
Province/State    307 non-null object
Country/Region    442 non-null object
Lat               442 non-null float64
Long              442 non-null float64
1/22/20           442 non-null int64
1/23/20           442 non-null int64
1/24/20           442 non-null int64
1/25/20           442 non-null int64
1/26/20           442 non-null int64
1/27/20           442 non-null int64
1/28/20           442 non-null int64
1/29/20           442 non-null int64
1/30/20           442 non-null int64
1/31/20           442 non-null int64
2/1/20            442 non-null int64
2/2/20            442 non-null int64
2/3/20            442 non-null int64
2/4/20            442 non-null int64
2/5/20            442 non-null int64
2/6/20            442 non-null int64
2/7/20            442 non-null int64
2/8/20            442 non-null int64
2/9/20            442 non-null int64
2/10/20           442 non-null int64
2

I think I just need to work with the confirmed DataFrame. It's set up such that each row is a specific location, and the columns are different dates.



In [4]:
#creating template for dataframes to record various values

df_template = df_confirmed.copy()


df_template.head(), df_template.size

(  Province/State Country/Region      Lat      Long  1/22/20  1/23/20  1/24/20  \
 0            NaN       Thailand  15.0000  101.0000        2        3        5   
 1            NaN          Japan  36.0000  138.0000        2        1        2   
 2            NaN      Singapore   1.2833  103.8333        0        1        3   
 3            NaN          Nepal  28.1667   84.2500        0        0        0   
 4            NaN       Malaysia   2.5000  112.5000        0        0        0   
 
    1/25/20  1/26/20  1/27/20  ...  3/5/20  3/6/20  3/7/20  3/8/20  3/9/20  \
 0        7        8        8  ...      47      48      50      50      50   
 1        2        4        4  ...     360     420     461     502     511   
 2        3        4        5  ...     117     130     138     150     150   
 3        1        1        1  ...       1       1       1       1       1   
 4        3        4        4  ...      50      83      93      99     117   
 
    3/10/20  3/11/20  3/12/20  3/13/

In [5]:
# Creating template out of JHU table format

do_not_include = ['Province/State', 'Country/Region', 'Lat', 'Long']

for x in df_template.columns.values:
    if (x not in do_not_include):
        for y in range(df_template['1/22/20'].values.size):
            df_template.at[y, x] = 0
            
            
df_template.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20
0,NaN,Thailand,15.0000,101.0000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,NaN,Japan,36.0000,138.0000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,NaN,Singapore,1.2833,103.8333,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,NaN,Nepal,28.1667,84.2500,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,NaN,Malaysia,2.5000,112.5000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
# Adding the rest of the beginning of January

for q in range(1, 22):
    day_str = str(q)
    date_str = '1/' + day_str + '/20'
    df_template[date_str] = 0    

df_template.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,1/12/20,1/13/20,1/14/20,1/15/20,1/16/20,1/17/20,1/18/20,1/19/20,1/20/20,1/21/20
0,NaN,Thailand,15.0000,101.0000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,NaN,Japan,36.0000,138.0000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,NaN,Singapore,1.2833,103.8333,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,NaN,Nepal,28.1667,84.2500,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,NaN,Malaysia,2.5000,112.5000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
df_template.columns.values

array(['Province/State', 'Country/Region', 'Lat', 'Long', '1/22/20',
       '1/23/20', '1/24/20', '1/25/20', '1/26/20', '1/27/20', '1/28/20',
       '1/29/20', '1/30/20', '1/31/20', '2/1/20', '2/2/20', '2/3/20',
       '2/4/20', '2/5/20', '2/6/20', '2/7/20', '2/8/20', '2/9/20',
       '2/10/20', '2/11/20', '2/12/20', '2/13/20', '2/14/20', '2/15/20',
       '2/16/20', '2/17/20', '2/18/20', '2/19/20', '2/20/20', '2/21/20',
       '2/22/20', '2/23/20', '2/24/20', '2/25/20', '2/26/20', '2/27/20',
       '2/28/20', '2/29/20', '3/1/20', '3/2/20', '3/3/20', '3/4/20',
       '3/5/20', '3/6/20', '3/7/20', '3/8/20', '3/9/20', '3/10/20',
       '3/11/20', '3/12/20', '3/13/20', '3/14/20', '1/1/20', '1/2/20',
       '1/3/20', '1/4/20', '1/5/20', '1/6/20', '1/7/20', '1/8/20',
       '1/9/20', '1/10/20', '1/11/20', '1/12/20', '1/13/20', '1/14/20',
       '1/15/20', '1/16/20', '1/17/20', '1/18/20', '1/19/20', '1/20/20',
       '1/21/20'], dtype=object)

In [8]:
ordered_columns = ['Province/State', 'Country/Region', 'Lat', 'Long', '1/1/20',
                  '1/2/20', '1/3/20', '1/4/20', '1/5/20', '1/6/20',
                   '1/7/20', '1/8/20', '1/9/20', '1/10/20', '1/11/20',
                   '1/12/20', '1/13/20', '1/14/20', '1/15/20', '1/16/20',
                   '1/17/20', '1/18/20', '1/19/20', '1/20/20', '1/21/20',
                   '1/22/20', '1/23/20', '1/24/20', '1/25/20', '1/26/20',
                   '1/27/20', '1/28/20', '1/29/20', '1/30/20', '1/31/20',
                   '2/1/20', '2/2/20', '2/3/20', '2/4/20', '2/5/20',
                   '2/6/20', '2/7/20', '2/8/20', '2/9/20', '2/10/20',
                   '2/11/20', '2/12/20', '2/13/20', '2/14/20', '2/15/20',
                   '2/16/20', '2/17/20', '2/18/20', '2/19/20', '2/20/20',
                   '2/21/20', '2/22/20', '2/23/20', '2/24/20', '2/25/20',
                   '2/26/20', '2/27/20', '2/28/20', '2/29/20', '3/1/20',
                   '3/2/20', '3/3/20', '3/4/20', '3/5/20', '3/6/20',
                   '3/7/20', '3/8/20', '3/9/20', '3/10/20', '3/11/20',
                  '3/12/20', '3/13/20', '3/14/20']

df_template = df_template[ordered_columns]

df_template.head()

,Province/State,Country/Region,Lat,Long,1/1/20,1/2/20,1/3/20,1/4/20,1/5/20,1/6/20,...,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20
0,NaN,Thailand,15.0000,101.0000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,NaN,Japan,36.0000,138.0000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,NaN,Singapore,1.2833,103.8333,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,NaN,Nepal,28.1667,84.2500,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,NaN,Malaysia,2.5000,112.5000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
df_template.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 442 entries, 0 to 441
Data columns (total 78 columns):
Province/State    307 non-null object
Country/Region    442 non-null object
Lat               442 non-null float64
Long              442 non-null float64
1/1/20            442 non-null int64
1/2/20            442 non-null int64
1/3/20            442 non-null int64
1/4/20            442 non-null int64
1/5/20            442 non-null int64
1/6/20            442 non-null int64
1/7/20            442 non-null int64
1/8/20            442 non-null int64
1/9/20            442 non-null int64
1/10/20           442 non-null int64
1/11/20           442 non-null int64
1/12/20           442 non-null int64
1/13/20           442 non-null int64
1/14/20           442 non-null int64
1/15/20           442 non-null int64
1/16/20           442 non-null int64
1/17/20           442 non-null int64
1/18/20           442 non-null int64
1/19/20           442 non-null int64
1/20/20           442 non-null int64
1

In [10]:
# For description of the different units being captured from the API,
# please see https://darksky.net/dev/docs
# Requests are being made in SI units

# dataframe to record median maximum temperature
df_tMax = df_template.copy()

# dataframe to record median minimum temperature
df_tMin = df_template.copy()

# dataframe to record median humidity
df_humidity = df_template.copy()

# dataframe to record median UV index
df_uvIndex = df_template.copy()

# dataframe to record median cloud cover
df_cloud = df_template.copy()

# dataframe to record weather summaries
#df_summaries = df_template.copy()

# dataframe to record precipitation probability
df_precipprob = df_template.copy()

# dataframe to record dewpoint
df_dewpoint = df_template.copy()

# dataframe to record pressure
df_pressure = df_template.copy()

# dataframe to record windspeed
df_windspeed = df_template.copy()

# dataframe to record "ozone" reading
df_ozone = df_template.copy()

# dataframe to record sunrise time
df_sunrise = df_template.copy()

# dataframe to record sunset time
df_sunset = df_template.copy()

In [24]:
#Darksky API key
API_KEY = 'API KEY'

#number of seconds in one day
one_day = 86400

t = '1583625600'

latitude = '31.8257'
longitude = '117.2264'

# sample get request 
#GET https://api.darksky.net/forecast/0123456789abcdef9876543210fedcba/
#42.3601,-71.0589,255657600?exclude=currently,flags

url = 'https://api.darksky.net/forecast/' + API_KEY + '/' + latitude + "," + longitude + ',' + t

url = url + '?exclude=currently,flags&units=si'
url

'https://api.darksky.net/forecast/0dbf1141c37674b871ee3d76c9593f31/31.8257,117.2264,1583625600?exclude=currently,flags&units=si'

In [25]:
response = requests.get(url)

In [26]:
response.content

b'{"latitude":31.8257,"longitude":117.2264,"timezone":"Asia/Shanghai","hourly":{"summary":"Possible light rain throughout the day.","icon":"rain","data":[{"time":1583596800,"summary":"Overcast","icon":"cloudy","precipIntensity":0,"precipProbability":0,"temperature":11.47,"apparentTemperature":11.47,"dewPoint":8.95,"humidity":0.85,"pressure":1015.9,"windSpeed":2.76,"windGust":4.26,"windBearing":105,"cloudCover":1,"uvIndex":0,"visibility":16.093,"ozone":326.1},{"time":1583600400,"summary":"Overcast","icon":"cloudy","precipIntensity":0,"precipProbability":0,"temperature":10.94,"apparentTemperature":10.94,"dewPoint":8.72,"humidity":0.86,"pressure":1015.4,"windSpeed":2.86,"windGust":5.08,"windBearing":100,"cloudCover":1,"uvIndex":0,"visibility":16.093,"ozone":324},{"time":1583604000,"summary":"Overcast","icon":"cloudy","precipIntensity":0,"precipProbability":0,"temperature":10.54,"apparentTemperature":10.54,"dewPoint":8.48,"humidity":0.87,"pressure":1014.6,"windSpeed":3.12,"windGust":6.05,"

In [27]:
info = json.loads(response.content)

info['daily']['data'][0]

{'time': 1583596800,
 'summary': 'Light rain throughout the day.',
 'icon': 'rain',
 'sunriseTime': 1583620260,
 'sunsetTime': 1583662500,
 'moonPhase': 0.45,
 'precipIntensity': 0.1234,
 'precipIntensityMax': 0.6269,
 'precipIntensityMaxTime': 1583636580,
 'precipProbability': 0.84,
 'precipType': 'rain',
 'temperatureHigh': 15.24,
 'temperatureHighTime': 1583653620,
 'temperatureLow': 10.28,
 'temperatureLowTime': 1583712000,
 'apparentTemperatureHigh': 14.96,
 'apparentTemperatureHighTime': 1583653620,
 'apparentTemperatureLow': 10.55,
 'apparentTemperatureLowTime': 1583712000,
 'dewPoint': 9.72,
 'humidity': 0.83,
 'pressure': 1012.1,
 'windSpeed': 5.19,
 'windGust': 12.34,
 'windGustTime': 1583624700,
 'windBearing': 98,
 'cloudCover': 0.61,
 'uvIndex': 6,
 'uvIndexTime': 1583639940,
 'visibility': 15.901,
 'ozone': 319.4,
 'temperatureMin': 9.14,
 'temperatureMinTime': 1583611200,
 'temperatureMax': 15.24,
 'temperatureMaxTime': 1583653620,
 'apparentTemperatureMin': 7.1,
 'appar

In [14]:
df_tMax.head()

,Province/State,Country/Region,Lat,Long,1/1/20,1/2/20,1/3/20,1/4/20,1/5/20,1/6/20,...,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20
0,NaN,Thailand,15.0000,101.0000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,NaN,Japan,36.0000,138.0000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,NaN,Singapore,1.2833,103.8333,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,NaN,Nepal,28.1667,84.2500,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,NaN,Malaysia,2.5000,112.5000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
# pull data from darksky weather API

# Columns to be skipped when iterating through the DataFrame
do_not_include = ['Province/State', 'Country/Region', 'Lat', 'Long']

#Darksky API key
API_KEY = 'API KEY'

# Number of seconds in one day
one_day = 86400


# Start iterating through the date columns
for x in df_tMax.columns.values:
    
    # Skip the columns that are not dates
    if (x not in do_not_include):
        
        # Create Unix time stamp out of the date column
        t = pd.to_datetime(df_tMax[x].name)
        t = int(t.value / 10**9)
        t = str(t)
                
        # Start iterating through the rows (locations)
        for y in range(df_tMax['1/22/20'].values.size):
            
                      
            # latitude and longitude coordinates of the row to be passed to the API            
            latitude = str(df_tMax.iloc[y][2])
            longitude = str(df_tMax.iloc[y][3])
                        
            # Building the URL for the API get
            url = 'https://api.darksky.net/forecast/' + API_KEY + '/' + latitude + "," + longitude + ',' + t
            url = url + '?exclude=currently,flags&units=si'
                
            # Getting the API call
            response = requests.get(url)
                
            # Putting the API response into the JSON thing
            info = json.loads(response.content)
            # Making a variable to more easily acccess JSON response data
            easy_info = info['daily']['data'][0]
                
            # Reading the JSON data
            tMax = easy_info['temperatureHigh']
            tMin = easy_info['temperatureLow']
            hum = easy_info['humidity'] * 100
            uvee = easy_info['uvIndex']
            clouds = easy_info['cloudCover'] * 100
            precip = easy_info['precipProbability'] * 100
            dew = easy_info['dewPoint']
            pressure = easy_info['pressure']
            wind = easy_info['windSpeed']
            ozone = easy_info['ozone']
            sunrise = easy_info['sunriseTime']
            sunset = easy_info['sunsetTime']
            
            # Recording the data into the respective dataframes
            df_tMax.at[y, x] = tMax
            df_tMin.at[y, x] = tMin
            df_humidity.at[y, x] = hum
            df_uvIndex.at[y, x] = uvee
            df_cloud.at[y, x] = clouds
            df_precipprob.at[y, x] = precip
            df_dewpoint.at[y, x] = dew
            df_pressure.at[y, x] = pressure
            df_windspeed.at[y, x] = wind
            df_ozone.at[y, x] = ozone
            df_sunrise.at[y,x] = sunrise
            df_sunset.at[y,x] = sunset

          
            # API has a daily limit, so we're just taking a break here
            if (y == 10):
                break
                
    if (x == '1/22/20'):
        break


In [12]:
df_tMax.head(12)

,Province/State,Country/Region,Lat,Long,1/1/20,1/2/20,1/3/20,1/4/20,1/5/20,1/6/20,...,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20
0,NaN,Thailand,15.0000,101.0000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,NaN,Japan,36.0000,138.0000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,NaN,Singapore,1.2833,103.8333,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,NaN,Nepal,28.1667,84.2500,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,NaN,Malaysia,2.5000,112.5000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,British Columbia,Canada,49.2827,-123.1207,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,New South Wales,Australia,-33.8688,151.2093,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Victoria,Australia,-37.8136,144.9631,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Queensland,Australia,-28.0167,153.4000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,NaN,Cambodia,11.5500,104.9167,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
# test run

# Columns to be skipped when iterating through the DataFrame
do_not_include = ['Province/State', 'Country/Region', 'Lat', 'Long']

#Darksky API key
API_KEY = 'API KEY'

# Start iterating through the date columns
for x in df_tMax.columns.values:
    
    # Skip the columns that are not dates
    if (x not in do_not_include):
        
        # Create Unix time stamp out of the date column
        t = pd.to_datetime(df_tMax[x].name)
        t = int(t.value / 10**9)
        t = str(t)
                
        # Start iterating through the rows (locations)
        for y in range(df_tMax['1/22/20'].values.size):
            
                      
            # latitude and longitude coordinates of the row to be passed to the API            
            latitude = str(df_tMax.iloc[y][2])
            longitude = str(df_tMax.iloc[y][3])
                        
            # Building the URL for the API get
            #url = 'https://api.darksky.net/forecast/' + API_KEY + '/' + latitude + "," + longitude + ',' + t
            #url = url + '?exclude=currently,flags&units=si'
                
            # Getting the API call
            #response = requests.get(url)
                
            # Putting the API response into the JSON thing
            #info = json.loads(response.content)
            # Making a variable to more easily acccess JSON response data
            #easy_info = info['daily']['data'][0]
                
            # Reading the JSON data
            tMax = y + y
            tMin = y + y
            hum = y + y
            uvee = y + y
            clouds = y + y
            precip = y + y
            dew = y + y
            pressure = y + y
            wind = y + y
            ozone = y + y
            sunrise = y + y
            sunset = y + y
            
            # Recording the data into the respective dataframes
            df_tMax.at[y, x] = tMax
            df_tMin.at[y, x] = tMin
            df_humidity.at[y, x] = hum
            df_uvIndex.at[y, x] = uvee
            df_cloud.at[y, x] = clouds
            df_precipprob.at[y, x] = precip
            df_dewpoint.at[y, x] = dew
            df_pressure.at[y, x] = pressure
            df_windspeed.at[y, x] = wind
            df_ozone.at[y, x] = ozone
            df_sunrise.at[y,x] = sunrise
            df_sunset.at[y,x] = sunset
            
            # API has a daily limit, so we're just taking a break here
            #if (y == 10):
             #   break
                
    #if (x == '1/22/20'):
     #   break


In [11]:
df_sunset.tail(12)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,1/12/20,1/13/20,1/14/20,1/15/20,1/16/20,1/17/20,1/18/20,1/19/20,1/20/20,1/21/20
335,"Prince George, MD",US,38.7849,-76.8721,670,670,670,670,670,670,...,670,670,670,670,670,670,670,670,670,670
336,"Pottawattamie, IA",US,41.3912,-95.4778,672,672,672,672,672,672,...,672,672,672,672,672,672,672,672,672,672
337,"Camden, SC",US,34.2465,-80.6070,674,674,674,674,674,674,...,674,674,674,674,674,674,674,674,674,674
338,"Pima, AZ",US,32.0575,-111.6661,676,676,676,676,676,676,...,676,676,676,676,676,676,676,676,676,676
339,"Noble, IN",US,41.4277,-85.3550,678,678,678,678,678,678,...,678,678,678,678,678,678,678,678,678,678
340,"Adams, IN",US,39.8522,-77.2865,680,680,680,680,680,680,...,680,680,680,680,680,680,680,680,680,680
341,"Boone, IN",US,40.0106,-86.4997,682,682,682,682,682,682,...,682,682,682,682,682,682,682,682,682,682
342,"Dane, WI",US,43.0186,-89.5498,684,684,684,684,684,684,...,684,684,684,684,684,684,684,684,684,684
343,"Pierce, WI",US,44.7509,-92.3814,686,686,686,686,686,686,...,686,686,686,686,686,686,686,686,686,686
344,"Cuyahoga, OH",US,41.4339,-81.6758,688,688,688,688,688,688,...,688,688,688,688,688,688,688,688,688,688


In [11]:
# The real API run

# Columns to be skipped when iterating through the DataFrame
do_not_include = ['Province/State', 'Country/Region', 'Lat', 'Long']

#Darksky API key
API_KEY = '33c9b1bcbb0ed583df2ce54549847848'

# Setting a counter for API calls as an emergency break, in case the thing gets out of control
# counter = 0

# Start iterating through the date columns
for x in df_tMax.columns.values:
    
    # Skip the columns that are not dates
    if (x not in do_not_include):
        
        # Create Unix time stamp out of the date column
        t = pd.to_datetime(df_tMax[x].name)
        t = int(t.value / 10**9)
        t = str(t)
                
        # Start iterating through the rows (locations)
        for y in range(df_tMax['1/22/20'].values.size):
            
                      
            # latitude and longitude coordinates of the row to be passed to the API            
            latitude = str(df_tMax.iloc[y][2])
            longitude = str(df_tMax.iloc[y][3])
                        
            # Building the URL for the API get
            url = 'https://api.darksky.net/forecast/' + API_KEY + '/' + latitude + "," + longitude + ',' + t
            url = url + '?exclude=currently,flags&units=si'
                
            # Getting the API call
            response = requests.get(url)
            # Incrementing the counter
            # counter = counter + 1
                
            # Putting the API response into the JSON thing
            info = json.loads(response.content)
            # Making a variable to more easily acccess JSON response data
            easy_info = info['daily']['data'][0]
                
            # Reading the JSON data
            tMax = easy_info['temperatureHigh']
            tMin = easy_info['temperatureLow']
            hum = easy_info['humidity'] * 100
            uvee = easy_info['uvIndex']
            clouds = easy_info['cloudCover'] * 100
            precip = easy_info['precipProbability'] * 100
            dew = easy_info['dewPoint']
            pressure = easy_info['pressure']
            wind = easy_info['windSpeed']
            ozone = easy_info['ozone']
            sunrise = easy_info['sunriseTime']
            sunset = easy_info['sunsetTime']
            
            # Recording the data into the respective dataframes
            df_tMax.at[y, x] = tMax
            df_tMin.at[y, x] = tMin
            df_humidity.at[y, x] = hum
            df_uvIndex.at[y, x] = uvee
            df_cloud.at[y, x] = clouds
            df_precipprob.at[y, x] = precip
            df_dewpoint.at[y, x] = dew
            df_pressure.at[y, x] = pressure
            df_windspeed.at[y, x] = wind
            df_ozone.at[y, x] = ozone
            df_sunrise.at[y,x] = sunrise
            df_sunset.at[y,x] = sunset
            
            # API has a daily limit, so we're just taking a break here
            #if (y == 3):
             #   break
                
    # By my calculation, this should run 17340 times, so I set the counter break at that number + 10, for a little buffer
    # in case I've screwed something up
    #if (x == '1/22/20'):
     #   break
     

In [14]:
df_sunrise.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,1/12/20,1/13/20,1/14/20,1/15/20,1/16/20,1/17/20,1/18/20,1/19/20,1/20/20,1/21/20
0,Anhui,Mainland China,31.8257,117.2264,1579648260,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Beijing,Mainland China,40.1824,116.4142,1579649580,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
2,Chongqing,Mainland China,30.0572,107.8740,1579650300,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
3,Fujian,Mainland China,26.0789,117.9874,1579647420,6,6,6,6,6,...,6,6,6,6,6,6,6,6,6,6
4,Gansu,Mainland China,36.0611,103.8343,8,8,8,8,8,8,...,8,8,8,8,8,8,8,8,8,8


In [12]:
df_tMax.head(12)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20
0,Anhui,Mainland China,31.82570,117.2264,7,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Beijing,Mainland China,40.18240,116.4142,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Chongqing,Mainland China,30.05720,107.8740,12,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Fujian,Mainland China,26.07890,117.9874,15,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Gansu,Mainland China,36.06110,103.8343,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Guangdong,Mainland China,23.34170,113.4244,21,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Guangxi,Mainland China,23.82980,108.7881,12,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Guizhou,Mainland China,26.81540,106.8748,7,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Hainan,Mainland China,19.19590,109.7453,23,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Hebei,Mainland China,38.04280,114.5149,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
df_tMin.head(12)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20
0,Anhui,Mainland China,31.82570,117.2264,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Beijing,Mainland China,40.18240,116.4142,-7,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Chongqing,Mainland China,30.05720,107.8740,6,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Fujian,Mainland China,26.07890,117.9874,5,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Gansu,Mainland China,36.06110,103.8343,-7,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Guangdong,Mainland China,23.34170,113.4244,12,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Guangxi,Mainland China,23.82980,108.7881,8,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Guizhou,Mainland China,26.81540,106.8748,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Hainan,Mainland China,19.19590,109.7453,14,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Hebei,Mainland China,38.04280,114.5149,-5,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
df_humidity.head(12)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20
0,Anhui,Mainland China,31.82570,117.2264,80,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Beijing,Mainland China,40.18240,116.4142,44,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Chongqing,Mainland China,30.05720,107.8740,76,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Fujian,Mainland China,26.07890,117.9874,79,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Gansu,Mainland China,36.06110,103.8343,60,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Guangdong,Mainland China,23.34170,113.4244,73,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Guangxi,Mainland China,23.82980,108.7881,83,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Guizhou,Mainland China,26.81540,106.8748,82,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Hainan,Mainland China,19.19590,109.7453,88,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Hebei,Mainland China,38.04280,114.5149,70,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
df_uvIndex.head(12)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20
0,Anhui,Mainland China,31.82570,117.2264,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Beijing,Mainland China,40.18240,116.4142,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Chongqing,Mainland China,30.05720,107.8740,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Fujian,Mainland China,26.07890,117.9874,5,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Gansu,Mainland China,36.06110,103.8343,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Guangdong,Mainland China,23.34170,113.4244,5,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Guangxi,Mainland China,23.82980,108.7881,4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Guizhou,Mainland China,26.81540,106.8748,4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Hainan,Mainland China,19.19590,109.7453,7,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Hebei,Mainland China,38.04280,114.5149,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
df_cloud.head(12)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20
0,Anhui,Mainland China,31.82570,117.2264,96,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Beijing,Mainland China,40.18240,116.4142,26,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Chongqing,Mainland China,30.05720,107.8740,83,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Fujian,Mainland China,26.07890,117.9874,74,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Gansu,Mainland China,36.06110,103.8343,67,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Guangdong,Mainland China,23.34170,113.4244,73,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Guangxi,Mainland China,23.82980,108.7881,96,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Guizhou,Mainland China,26.81540,106.8748,96,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Hainan,Mainland China,19.19590,109.7453,48,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Hebei,Mainland China,38.04280,114.5149,44,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
df_precipprob.head(12)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20
0,Anhui,Mainland China,31.82570,117.2264,40,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Beijing,Mainland China,40.18240,116.4142,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Chongqing,Mainland China,30.05720,107.8740,28,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Fujian,Mainland China,26.07890,117.9874,9,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Gansu,Mainland China,36.06110,103.8343,7,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Guangdong,Mainland China,23.34170,113.4244,13,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Guangxi,Mainland China,23.82980,108.7881,25,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Guizhou,Mainland China,26.81540,106.8748,22,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Hainan,Mainland China,19.19590,109.7453,10,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Hebei,Mainland China,38.04280,114.5149,5,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
df_dewpoint.head(12)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20
0,Anhui,Mainland China,31.82570,117.2264,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Beijing,Mainland China,40.18240,116.4142,-13,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Chongqing,Mainland China,30.05720,107.8740,4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Fujian,Mainland China,26.07890,117.9874,5,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Gansu,Mainland China,36.06110,103.8343,-9,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Guangdong,Mainland China,23.34170,113.4244,12,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Guangxi,Mainland China,23.82980,108.7881,7,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Guizhou,Mainland China,26.81540,106.8748,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Hainan,Mainland China,19.19590,109.7453,15,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Hebei,Mainland China,38.04280,114.5149,-6,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
df_pressure.head(12)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20
0,Anhui,Mainland China,31.82570,117.2264,1026,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Beijing,Mainland China,40.18240,116.4142,1027,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Chongqing,Mainland China,30.05720,107.8740,1020,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Fujian,Mainland China,26.07890,117.9874,1021,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Gansu,Mainland China,36.06110,103.8343,1023,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Guangdong,Mainland China,23.34170,113.4244,1019,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Guangxi,Mainland China,23.82980,108.7881,1020,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Guizhou,Mainland China,26.81540,106.8748,1019,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Hainan,Mainland China,19.19590,109.7453,1015,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Hebei,Mainland China,38.04280,114.5149,1027,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
df_windspeed.head(12)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20
0,Anhui,Mainland China,31.82570,117.2264,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Beijing,Mainland China,40.18240,116.4142,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Chongqing,Mainland China,30.05720,107.8740,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Fujian,Mainland China,26.07890,117.9874,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Gansu,Mainland China,36.06110,103.8343,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Guangdong,Mainland China,23.34170,113.4244,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Guangxi,Mainland China,23.82980,108.7881,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Guizhou,Mainland China,26.81540,106.8748,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Hainan,Mainland China,19.19590,109.7453,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Hebei,Mainland China,38.04280,114.5149,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
df_sunrise.head(12)

,Province/State,Country/Region,Lat,Long,1/1/20,1/2/20,1/3/20,1/4/20,1/5/20,1/6/20,...,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20
0,NaN,Thailand,15.0000,101.0000,1577835780,1577922180,1578008580,1578095040,1578181440,1578267840,...,1583191980,1583278320,1583364720,1583451060,1583537400,1583623800,1583710140,1583796480,1583882880,1583969220
1,NaN,Japan,36.0000,138.0000,1577829600,1577916000,1578002400,1578088800,1578175200,1578261600,...,1583183820,1583270100,1583356440,1583442780,1583529060,1583615400,1583701680,1583788020,1583874360,1583960640
2,NaN,Singapore,1.2833,103.8333,1577833680,1577920080,1578006540,1578092940,1578179340,1578265800,...,1583190900,1583277300,1583363700,1583450040,1583536440,1583622840,1583709240,1583795580,1583881980,1583968380
3,NaN,Nepal,28.1667,84.2500,1577841360,1577927760,1578014160,1578100620,1578187020,1578273420,...,1583196420,1583282760,1583369100,1583455440,1583541720,1583628060,1583714400,1583800740,1583887080,1583973420
4,NaN,Malaysia,2.5000,112.5000,1577831700,1577918100,1578004560,1578090960,1578177420,1578263820,...,1583188860,1583275260,1583361660,1583448000,1583534400,1583620800,1583707140,1583793540,1583879940,1583966280
5,British Columbia,Canada,49.2827,-123.1207,1577808540,1577894940,1577981340,1578067740,1578154140,1578240480,...,1583160780,1583247000,1583333280,1583419560,1583505840,1583592120,1583678400,1583764680,1583850960,1583937240
6,New South Wales,Australia,-33.8688,151.2093,1577818140,1577904540,1577991000,1578077460,1578163920,1578250320,...,1583178360,1583264820,1583351280,1583437740,1583524200,1583610600,1583697060,1583783520,1583869980,1583956440
7,Victoria,Australia,-37.8136,144.9631,1577818920,1577905380,1577991840,1578078300,1578164760,1578251220,...,1583179680,1583266140,1583352600,1583439060,1583525520,1583611980,1583698440,1583784900,1583871360,1583957820
8,Queensland,Australia,-28.0167,153.4000,1577818440,1577904900,1577991360,1578077760,1578164220,1578250680,...,1583178120,1583264520,1583350980,1583437380,1583523840,1583610300,1583696700,1583783160,1583869560,1583956020
9,NaN,Cambodia,11.5500,104.9167,1577834460,1577920860,1578007320,1578093720,1578180120,1578266580,...,1583190960,1583277300,1583363700,1583450040,1583536380,1583622780,1583709120,1583795520,1583881860,1583968260


In [12]:
# writing CSVs of what I've got so far

df_tMax.to_csv('./csv/tMax.csv', index=False)

df_tMin.to_csv('./csv/tMin.csv', index=False)

df_humidity.to_csv('./csv/humidity.csv', index=False)

df_uvIndex.to_csv('./csv/uv.csv', index=False)

df_cloud.to_csv('./csv/cloud.csv', index=False)

df_precipprob.to_csv('./csv/precip.csv', index=False)

df_dewpoint.to_csv('./csv/dew.csv', index=False)

df_pressure.to_csv('./csv/pressure.csv', index=False)

df_windspeed.to_csv('./csv/wind.csv', index=False)

df_ozone.to_csv('./csv/ozone.csv', index=False)

df_sunrise.to_csv('./csv/sunrise.csv', index=False)

df_sunset.to_csv('./csv/sunset.csv', index=False)
